In [1]:
import pandas as pd
#Scraping With GetOldTweets3
#!pip install GetOldTweets3
import datetime as dt
import GetOldTweets3 as got
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
import textblob as TextBlob
from bs4 import BeautifulSoup
from nltk.stem import PorterStemmer


In [2]:
keyword = 'stockmarket'
old_date = '2020-01-01'
new_date = '2020-03-31'
number_tweets = 800

In [3]:
def get_stock_tweet(old_date,new_date):
    tweet_criteria = got.manager.TweetCriteria().setQuerySearch(keyword)\
                                            .setSince(old_date)\
                                            .setUntil(new_date)\
                                            .setTopTweets(True)\
                                            .setMaxTweets(number_tweets)
    return(got.manager.TweetManager.getTweets(tweet_criteria))

In [4]:
import datetime as dt
#start_date = dt.datetime(2020, 01,01)
start_date_str = "03/31/20"
end_date_str='01/01/20'
start_date = dt.datetime.strptime(start_date_str, "%m/%d/%y")
end_date=dt.datetime.strptime(end_date_str, "%m/%d/%y")
all_tweets =[]
while start_date > end_date:
    tmp_date = start_date - dt.timedelta(days=7)
    print("start_date:",start_date," end_date",tmp_date)
    
    all_tweets.extend(get_stock_tweet(tmp_date.strftime("%Y-%m-%d"),start_date.strftime("%Y-%m-%d")))
    start_date=tmp_date

start_date: 2020-03-31 00:00:00  end_date 2020-03-24 00:00:00
start_date: 2020-03-24 00:00:00  end_date 2020-03-17 00:00:00
start_date: 2020-03-17 00:00:00  end_date 2020-03-10 00:00:00
start_date: 2020-03-10 00:00:00  end_date 2020-03-03 00:00:00
start_date: 2020-03-03 00:00:00  end_date 2020-02-25 00:00:00
start_date: 2020-02-25 00:00:00  end_date 2020-02-18 00:00:00
start_date: 2020-02-18 00:00:00  end_date 2020-02-11 00:00:00
start_date: 2020-02-11 00:00:00  end_date 2020-02-04 00:00:00
start_date: 2020-02-04 00:00:00  end_date 2020-01-28 00:00:00
start_date: 2020-01-28 00:00:00  end_date 2020-01-21 00:00:00
start_date: 2020-01-21 00:00:00  end_date 2020-01-14 00:00:00
start_date: 2020-01-14 00:00:00  end_date 2020-01-07 00:00:00
start_date: 2020-01-07 00:00:00  end_date 2019-12-31 00:00:00


In [5]:
tweet_df = pd.DataFrame({'got_criteria':all_tweets})
tweet_df

,got_criteria
0,<GetOldTweets3.models.Tweet.Tweet object at 0x...
1,<GetOldTweets3.models.Tweet.Tweet object at 0x...
2,<GetOldTweets3.models.Tweet.Tweet object at 0x...
3,<GetOldTweets3.models.Tweet.Tweet object at 0x...
4,<GetOldTweets3.models.Tweet.Tweet object at 0x...
...,...
2446,<GetOldTweets3.models.Tweet.Tweet object at 0x...
2447,<GetOldTweets3.models.Tweet.Tweet object at 0x...
2448,<GetOldTweets3.models.Tweet.Tweet object at 0x...
2449,<GetOldTweets3.models.Tweet.Tweet object at 0x...


In [6]:
tweet_df["tweet_text"] = tweet_df["got_criteria"].apply(lambda x: x.text)
tweet_df["tweet_text"]

tweet_df["date"] = tweet_df["got_criteria"].apply(lambda x: x.date)
tweet_df["date"]

tweet_df["hashtags"] = tweet_df["got_criteria"].apply(lambda x: x.hashtags)
tweet_df["hashtags"]

tweet_df["link"] = tweet_df["got_criteria"].apply(lambda x: x.permalink)
#tweet_df["link"]

In [7]:
#tweet_df
from datetime import datetime as dt
tweet_df['date'] = tweet_df['date'].dt.normalize()
tweet_df['date'] = tweet_df['date'].dt.strftime('%Y-%m-%d')
tweet_df

,got_criteria,tweet_text,date,hashtags,link
0,<GetOldTweets3.models.Tweet.Tweet object at 0x...,"""Liquidity shall never freeze again"" ICE9 -> S...",2020-03-30,,https://twitter.com/Paisan26849860/status/1244...
1,<GetOldTweets3.models.Tweet.Tweet object at 0x...,"Mar 30: S&P 500 UP +3.35%, NASDAQ +3.62%, DJIA...",2020-03-30,#SPX #NASDAQ #DJIA #stocks #StockMarket #COVID...,https://twitter.com/DrivingTheDay/status/12447...
2,<GetOldTweets3.models.Tweet.Tweet object at 0x...,The #stockmarket is detached from economic rea...,2020-03-30,#stockmarket,https://twitter.com/SchiffGold/status/12447436...
3,<GetOldTweets3.models.Tweet.Tweet object at 0x...,bUt ThE sToCkMarKet iS dOiNg weLL!,2020-03-30,,https://twitter.com/sohmer/status/124473691241...
4,<GetOldTweets3.models.Tweet.Tweet object at 0x...,An important message from one of the top #Saud...,2020-03-30,#Saudi #economic #fund #Corona #COVID19 #econo...,https://twitter.com/Salansar1/status/124473247...
...,...,...,...,...,...
2446,<GetOldTweets3.models.Tweet.Tweet object at 0x...,Lundin-Gold-Inc. is Top Pick at StockChase htt...,2019-12-31,#stocks #finance #stockmarket #StockPicks,https://twitter.com/stockchase/status/12117992...
2447,<GetOldTweets3.models.Tweet.Tweet object at 0x...,They r ramping up in Ecuador. He has be...Lund...,2019-12-31,#stocks #finance #stockmarket #StockPicks,https://twitter.com/stockchase/status/12117992...
2448,<GetOldTweets3.models.Tweet.Tweet object at 0x...,Turquoise-Hill-Resources is Top Pick at StockC...,2019-12-31,#stocks #finance #stockmarket #StockPicks,https://twitter.com/stockchase/status/12117992...
2449,<GetOldTweets3.models.Tweet.Tweet object at 0x...,FREE TRIAL! Ready for a green 2020??? Check ou...,2019-12-31,#trading #DayTrading #WallStreet #StockMarket ...,https://twitter.com/TeresaTrades/status/121179...


In [8]:
tweet_df.to_excel("tweet_data_master.xlsx")

/Users/priyankakushwaha/opt/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:931: UserWarning: Ignoring URL 'https://www.cnn.com/2020/03/28/politics/social-distancing-us-coronavirus-15-days-charts/index.html%20The%20record%20' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/Users/priyankakushwaha/opt/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:931: UserWarning: Ignoring URL 'https://www.tc2000.com/~hqgnLy%20' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/Users/priyankakushwaha/opt/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:931: UserWarning: Ignoring URL 'https://www.wired.com/story/coronavirus-interview-larry-brilliant-smallpox-epidemiologist/?utm_source=pocket-newtab%20The%20need%20to%20work%20for%20your%20self%20is%20even%20more%20so.%20Governments%20who%20fail%20to%20listen%20to%20the%20right%20people%20fail%20t

In [9]:
data = pd.read_excel("tweet_data_master.xlsx")
#data

In [10]:
data.tweet_text.fillna("", inplace=True)
data.tweet_text.dtype

dtype('O')

In [11]:
data['word_count'] = data['tweet_text'].apply(lambda x: len(str(x).split(" ")))
#data.head(10)

In [12]:
import string
data['no_punctuation'] = data['tweet_text'].str.replace('[^\w\s]','')
data.head()

,Unnamed: 0,got_criteria,tweet_text,date,hashtags,link,word_count,no_punctuation
0,0,<GetOldTweets3.models.Tweet.Tweet object at 0x...,"""Liquidity shall never freeze again"" ICE9 -> S...",2020-03-30,NaN,https://twitter.com/Paisan26849860/status/1244...,26,Liquidity shall never freeze again ICE9 Stock...
1,1,<GetOldTweets3.models.Tweet.Tweet object at 0x...,"Mar 30: S&P 500 UP +3.35%, NASDAQ +3.62%, DJIA...",2020-03-30,#SPX #NASDAQ #DJIA #stocks #StockMarket #COVID...,https://twitter.com/DrivingTheDay/status/12447...,20,Mar 30 SP 500 UP 335 NASDAQ 362 DJIA 319 https...
2,2,<GetOldTweets3.models.Tweet.Tweet object at 0x...,The #stockmarket is detached from economic rea...,2020-03-30,#stockmarket,https://twitter.com/SchiffGold/status/12447436...,8,The stockmarket is detached from economic real...
3,3,<GetOldTweets3.models.Tweet.Tweet object at 0x...,bUt ThE sToCkMarKet iS dOiNg weLL!,2020-03-30,NaN,https://twitter.com/sohmer/status/124473691241...,7,bUt ThE sToCkMarKet iS dOiNg weLL
4,4,<GetOldTweets3.models.Tweet.Tweet object at 0x...,An important message from one of the top #Saud...,2020-03-30,#Saudi #economic #fund #Corona #COVID19 #econo...,https://twitter.com/Salansar1/status/124473247...,41,An important message from one of the top Saudi...


In [13]:
tok = WordPunctTokenizer()
ps = PorterStemmer() 

pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
negations_dic = {"he’s":"he is","isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not", "might’ve": "might have","y’all":"you all","we're":"we are","you’re":"you "}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def clean_tweet(text):
    try:
        soup = BeautifulSoup(text,'lxml')
        souped = soup.get_text()
    except:
        souped = ""
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    lower_case = stripped.lower()
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    words_without_sw = [word for word in words if not word in stopwords.words()] # Remove stop words
    #print("words_without_sw")
    stem_words = [ps.stem(word) for word in words_without_sw] #convert to stem word
    #print("stem_words")
    return (" ".join(stem_words)).strip()
    

In [14]:
result = []

for tweet in data.tweet_text:
    result.append(clean_tweet(tweet))

data['cleaned_text'] = result

data.head()

,Unnamed: 0,got_criteria,tweet_text,date,hashtags,link,word_count,no_punctuation,cleaned_text
0,0,<GetOldTweets3.models.Tweet.Tweet object at 0x...,"""Liquidity shall never freeze again"" ICE9 -> S...",2020-03-30,NaN,https://twitter.com/Paisan26849860/status/1244...,26,Liquidity shall never freeze again ICE9 Stock...,liquid shall never freez ice stockmarket close...
1,1,<GetOldTweets3.models.Tweet.Tweet object at 0x...,"Mar 30: S&P 500 UP +3.35%, NASDAQ +3.62%, DJIA...",2020-03-30,#SPX #NASDAQ #DJIA #stocks #StockMarket #COVID...,https://twitter.com/DrivingTheDay/status/12447...,20,Mar 30 SP 500 UP 335 NASDAQ 362 DJIA 319 https...,nasdaq djia spx spi nasdaq djia stock stockmar...
2,2,<GetOldTweets3.models.Tweet.Tweet object at 0x...,The #stockmarket is detached from economic rea...,2020-03-30,#stockmarket,https://twitter.com/SchiffGold/status/12447436...,8,The stockmarket is detached from economic real...,stockmarket detach econom realiti
3,3,<GetOldTweets3.models.Tweet.Tweet object at 0x...,bUt ThE sToCkMarKet iS dOiNg weLL!,2020-03-30,NaN,https://twitter.com/sohmer/status/124473691241...,7,bUt ThE sToCkMarKet iS dOiNg weLL,stockmarket well
4,4,<GetOldTweets3.models.Tweet.Tweet object at 0x...,An important message from one of the top #Saud...,2020-03-30,#Saudi #economic #fund #Corona #COVID19 #econo...,https://twitter.com/Salansar1/status/124473247...,41,An important message from one of the top Saudi...,import messag top saudi economist privat secto...


In [16]:
data.to_excel("clean_tweet_data_master.xlsx")
